# This is a simplified version of my 18th place solution in the **Shopee - Price Match Guarantee** contest
## I replaced my image models with resnet18 to showcase that even a very basic model could do well and was enough to score a silver medal in this competition

# The outline of my approach

### Step 1 training 
I used arcface for all my training, and fine tune pretrained models with it for a few epochs. For image I tried resnet, efficient net and nfnet. The nfnet worked the best for me so that’s in my final solution. The resnet18 I used in the toy solution just to show a very basic model can work too. 

For text I found that language model pretrained on Indonesian language worked the best.

### Step 2 Embeddings and similarities 
For each model I first generate embedddings and then calculate the full cosine similarity matrix.

### Step 3 Combine the model outputs
I combine the matrices from the previous step with formula D = 1 - (1 - D<sub>TFIDF</sub>) * (1 - D<sub>Resnet</sub>) * (1 - D<sub>BERT</sub>)

This I found works much better than alternatives of taking mean or max.

### Step 4 Rerank
I replace the predictions of each row with an average of predictions of its nearest neighbors. I set threshold for “nearest” so that 4 neighbors are used on average (found experimentally)

It helps the score a bit.

### Step 5 force groups into a desired distribution 
This was a largest single change I made, jumping my score from .739 to .759
I nicknamed it “chiseling” the idea is: I first make an educated guess that the distribution of group targets in the test data is similar to the one in train. Then I make my solution to have the same shape.

I first decide that groups with 2 elements are going to be those where the third largest element is the lowest. Then I follow the same logic for all the sizes up to 50.

### No thresholds!
Early in the competition I followed the public approaches of selecting elements to pick based on the hard coded threshold - this had to be tuned for test data separately and eat into your submissions limit and time. (Especially with multiple models). When I moved to make decisions based on the data distribution instead, it made the code much more resilient with less need for tuning when changing/adding models.



In [ ]:
from fastai.vision.all import *
from tqdm.notebook import tqdm
import sklearn.feature_extraction.text
from transformers import (BertTokenizer, AutoConfig, AutoModel)
from sklearn.model_selection import StratifiedKFold

In [ ]:
PATH = Path('../input/shopee-product-matching')

BERT_PATH = '../input/bertindo15g'
bert_model_file = '../input/shopee-small-models/bert_indo_val0.pth'

image_model_file = '../input/shopee-small-models/resnet18val0.pth'


In [ ]:
# this is used for reranking, controls how many neighbours on average are considered
RECIPROCAL_PER_ROW = 4.2

## Bert Model

In [ ]:
class BertTextModel(nn.Module):
    def __init__(self, bert_model):
        super().__init__()
        self.bert_model = bert_model
    def forward(self, x):
        output = self.bert_model(*x)
        return output.last_hidden_state[:,0,:]

In [ ]:
def load_bert_model(fname):
    model = AutoModel.from_config(AutoConfig.from_pretrained(BERT_PATH))
    state = torch.load(fname)
    model.load_state_dict(state)
    return BertTextModel(model).cuda().eval()

### Dataloader for Bert

In [ ]:
#Taken from https://www.kaggle.com/c/shopee-product-matching/discussion/233605#1278984
def string_escape(s, encoding='utf-8'):
    return s.encode('latin1').decode('unicode-escape').encode('latin1').decode(encoding)

class TitleTransform(Transform):
    def __init__(self):
        super().__init__()
        self.tokenizer = BertTokenizer.from_pretrained(BERT_PATH)
               
    def encodes(self, row):
        text = row.title
        text=string_escape(text)
        encodings = self.tokenizer(text, padding = 'max_length', max_length=100, truncation=True,return_tensors='pt')
        keys =['input_ids', 'attention_mask', 'token_type_ids'] 
        return tuple(encodings[key].squeeze() for key in keys)

def get_text_dls():
    tfm = TitleTransform()

    data_block = DataBlock(
        blocks = (TransformBlock(type_tfms=tfm), 
                  CategoryBlock(vocab=train_df.label_group.to_list())),
        splitter=ColSplitter(),
        get_y=ColReader('label_group'),
        )
    return  data_block.dataloaders(train_df, bs=256)

## TFIDF

In [ ]:
def csr_matrix_to_tensor(csr):
    coo = csr.tocoo()
    t = torch.sparse_coo_tensor([coo.row, coo.col], coo.data, csr.shape).cuda()
    return t

def get_tfid_embs(data, idxs):
    sk_model = sklearn.feature_extraction.text.TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)
    text_embeddings =sk_model.fit_transform(data.title)
    text_embeddings =text_embeddings[idxs]
    return text_embeddings

def generate_tfid_D(text_embeddings, out):
    emb_size = text_embeddings.shape[0]
    
    sparse_embs = csr_matrix_to_tensor(text_embeddings)
    step = 100
    for chunk_start in range(0, emb_size, step):
        chunk_end = min(chunk_start+step, emb_size)
        chunk = text_embeddings[chunk_start:chunk_end]
        chunk = csr_matrix_to_tensor(chunk).to_dense()
        tmp = sparse_embs @ chunk.T
        tmp.clip_(0,1)
        out[chunk_start:chunk_end]=tmp.half().T

## IMAGE

In [ ]:
class ResnetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.body = create_body(resnet18, cut=-2, pretrained=False)
        self.after_conv=nn.Sequential(
            AdaptiveConcatPool2d(),
            Flatten(),
            nn.BatchNorm1d(1024)
        )
    def forward(self, x):
        x = self.body(x)
        return self.after_conv(x)
        

In [ ]:
def load_image_model(fname):
    state_dict = torch.load(fname)
    model = ResnetModel()
    model.load_state_dict(state_dict)
    model = model.eval().cuda()
    return model

In [ ]:
def get_img_file(row):
    img =row.image
    fn  = PATH/'train_images'/img
    if not fn.is_file():
        fn = PATH/'test_images'/img
    return fn

def get_image_dls(size, bs):
    data_block = DataBlock(blocks = (ImageBlock(), CategoryBlock(vocab=train_df.label_group.to_list())),
                 splitter=ColSplitter(),
                 get_y=ColReader('label_group'),
                 get_x=get_img_file,
                 item_tfms=Resize(size*2, resamples=(Image.BICUBIC,Image.BICUBIC)), 
                 batch_tfms=aug_transforms(size=size, min_scale=0.75)+[Normalize.from_stats(*imagenet_stats)],
                 )
    return data_block.dataloaders(train_df, bs=bs)



## Helper code

In [ ]:
# Code used in the solution

def gen_sim_and(embs_list, res):
    emb_size = len(embs_list[0])
    step = 100
    cache = torch.empty((step, emb_size), device = 'cuda', dtype=embs_list[0].dtype)
    for embs in embs_list:
        embs = embs.cuda()
        for chunk_start in range(0, emb_size, step):
            chunk_end = min(chunk_start+step, emb_size)
            chunk=embs[chunk_start: chunk_end]
            tmp = cache[:chunk_end-chunk_start]
            torch.matmul(chunk, embs.T, out = tmp)
            tmp.clip_(0,1)
            tmp.mul_(-1)
            tmp.add_(1)
            res[chunk_start:chunk_end].mul_(tmp)
    res.mul_(-1)
    res.add_(1)

def reciprocal_probs(D, x, thresh):
    neighb_cnt = torch.count_nonzero(D[x]>thresh)
    if neighb_cnt > 50:
        _,neighb= D[x].topk(50)
    else:
        neighb=torch.nonzero(D[x]>thresh) 
    DP = D[neighb]
    DP = DP.mean(dim=0)
    return DP

def find_threshold(D):
    k=D.numel()-int(RECIPROCAL_PER_ROW*len(D))
    threshold=D.view(-1).kthvalue(k).values
    return threshold

def rerank(D):
    if len(D)<4: threshold =0
    else: 
        threshold = find_threshold(D[:20000])
        print("threshold for rerank:", threshold.item())
    for i in range(len(D)):
        D[i]=reciprocal_probs(D, i,threshold)

def dist_to_edges(dist):
    res = []
    K = min(51, len(dist))
    for x in range(len(dist)):
        vals, ys = dist[x].topk(K)
        for v,y in zip(vals.tolist(),ys.tolist()):
            res.append((x,y,v))
    return sorted(res, key=lambda x: -x[2])

def get_group_probs(groups, D):
    group_probs=[]
    new_groups=[]
    for x in range(len(groups)):
        gr = groups[x]
        gr_probs = D[x][gr]
        with_prob =sorted(list(zip(gr, gr_probs)), key=lambda x: -x[1])
        new_groups.append([wp[0] for wp in with_prob])
        group_probs.append([wp[1]for wp in with_prob])
    return new_groups, group_probs

def edges_to_groups(edges, N):
    groups = [[] for i in range(N)]
    groups_p = [[] for _ in range(N)]
    for x,y,v in edges:
        if len(groups[x])>=51: continue
        groups[x].append(y)
        groups_p[x].append(v)
    return groups, groups_p

def add_target_groups(data_df, source_column='label_group', target_column='target'):
    target_groups = data_df.groupby(source_column).indices
    data_df[target_column]=data_df[source_column].map(target_groups)
    return data_df

def add_splits(train_df, valid_group=0):
    grouped = train_df.groupby('label_group').size()

    labels, sizes =grouped.index.to_list(), grouped.to_list()

    skf = StratifiedKFold(5)
    splits = list(skf.split(labels, sizes))

    group_to_split =  dict()
    for idx in range(5):
        labs = np.array(labels)[splits[idx][1]]
        group_to_split.update(dict(zip(labs, [idx]*len(labs))))

    train_df['split'] = train_df.label_group.replace(group_to_split)
    train_df['is_valid'] = train_df['split'] == valid_group
    return train_df

def embs_from_model(model, dl):
    all_embs = []
    all_ys=[]
    for batch in tqdm(dl):
        if len(batch) ==2:
            bx,by=batch
        else:
            bx,=batch
            by=torch.zeros(1)
        with torch.no_grad():
            embs = model(bx)
            all_embs.append(embs.half())
        all_ys.append(by)
    all_embs = F.normalize(torch.cat(all_embs))
    return all_embs, torch.cat(all_ys)

def get_targets_shape(train_df):
    all_targets = add_target_groups(train_df).target.to_list()
    all_targets_lens = [len(t) for t in all_targets]
    targets_shape = []
    for size in range(min(all_targets_lens), max(all_targets_lens)+1):
        count = all_targets_lens.count(size) / len(all_targets)
        targets_shape.append((size,count))
    return targets_shape

def chisel(groups, groups_p, pos, target_count):
    probs = []
    groups_lens = [len(g)for g in groups]
    current_count = groups_lens.count(pos)
    if current_count >= target_count:

        return
    to_cut = target_count - current_count
    for i in range(len(groups)):
        if len(groups_p[i])>pos:
            probs.append((i, groups_p[i][pos]))
    probs.sort(key=lambda x:x[1])
    for i in range(min(to_cut, len(probs))):
        group_idx = probs[i][0] 
        groups[group_idx]=groups[group_idx][:pos]
        groups_p[group_idx]=groups_p[group_idx][:pos]

In [ ]:
# Not used in solution, just for illustration purpose
def f1(tp, fp, num_tar):
    return 2 * tp / (tp+fp+num_tar)

def build_from_pairs(pairs, target, display = True):
    score =0
    tp = [0]*len(target)
    fp = [0]*len(target)
    scores=[]
    vs=[]
    group_sizes = [len(x) for x in target]
    for x, y, v in pairs:
        group_size = group_sizes[x]
        score -= f1(tp[x], fp[x], group_size)
        if y in target[x]: tp[x] +=1
        else: fp[x] +=1
        score += f1(tp[x], fp[x], group_size) 
        scores.append(score / len(target))
        vs.append(v)
    if display:
        plt.plot(scores)
        am =torch.tensor(scores).argmax()
        print(f'{scores[am]:.3f} at {am/len(target)} pairs or {vs[am]:.3f} threshold')
    return scores


def score_distances(dist, targets, display=False):
    triplets = dist_to_edges(dist)[:len(dist)*10]
    return max(build_from_pairs(triplets, targets, display))

def score_group(group, target):
    tp = len(set(group).intersection(set(target)))
    return 2 * tp / (len(group)+len(target))
def score_all_groups(groups, targets):
    scores = [score_group(groups[i], targets[i]) for i in range(len(groups))]
    return sum(scores)/len(scores)
def show_groups(groups, targets):
    groups_lens = [len(g)for g in groups]
    targets_lens = [len(g) for g in targets]
    plt.figure(figsize=(8,8)) 
    plt.hist((groups_lens,targets_lens) ,bins=list(range(1,52)), label=['preds', 'targets'])
    plt.legend()
    plt.title(f'score: {score_all_groups(groups, targets):.3f}')
    plt.show()

## Check on validation set

In [ ]:
train_df = pd.read_csv(PATH/'train.csv')
train_df = add_splits(train_df)

### Generating embeddings from Resnet and BERT

In [ ]:
img_embs,ys = embs_from_model(load_image_model(image_model_file), get_image_dls(224,256).valid)

bert_embs, ys = embs_from_model(load_bert_model(bert_model_file), get_text_dls().valid)

### Calculating all the similiraties and combining them together
The formula I use is D = 1 - (1 - D<sub>TFIDF</sub>) * (1 - D<sub>Resnet</sub>) * (1 - D<sub>BERT</sub>)
The reason the code looks more complicated is that I can only fit a single 70Kx70K fp16 matrix in the memory, that's why why I do multiplications chunk by chunk inside the `gen_sim_and` function. I also use pytorch inplace operators like `mul_` not to allocate any additional memory

In [ ]:
D = torch.empty((img_embs.shape[0], img_embs.shape[0]), device = 'cuda', dtype=torch.float16)
tfid_embs = get_tfid_embs(train_df, train_df[train_df.is_valid].index.tolist())
generate_tfid_D(tfid_embs,D)
D.mul_(-1)
D.add_(1)
gen_sim_and([img_embs,bert_embs],D)

### Here is the score if we just used a fixed Threshold
The chart shows the competition metric depending on the number of edges we include in the submission and prints the maximum value

In [ ]:
targets = [torch.where(t)[0].tolist() for t in ys[:,None]==ys[None,:]]
score_distances(D,targets, display=True)

### Here is the score after Reranking step:

In [ ]:
rerank(D)
score_distances(D,targets, display=True)

### And here are the sizes of all the groups compared to the ground truth

In [ ]:
edges = dist_to_edges(D)
groups, groups_p = edges_to_groups(edges[:6*len(D)], len(D))

show_groups(groups, targets)

### Here we perform the chisel step to match the two distributions
Note the improved score, this effect is more pronounced on the test set where the initial grouping is further apart from the target

In [ ]:
edges = dist_to_edges(D)
groups, groups_p = edges_to_groups(edges, len(D))
for pos, size_pct in get_targets_shape(train_df):
    chisel(groups, groups_p, pos, int(size_pct * len(groups)))
show_groups(groups, targets)

## Run on the test set
same as above, just without displaying stuff and deleting objects we no longer need to preserve memory

In [ ]:
test_df = pd.read_csv(PATH/'test.csv')

In [ ]:
bert_embs,_ = embs_from_model(load_bert_model(bert_model_file), get_text_dls().test_dl(test_df))

img_embs,_ =embs_from_model(load_image_model(image_model_file), get_image_dls(224, 256).test_dl(test_df))

In [ ]:
tfid_embs=  get_tfid_embs(pd.concat([test_df,train_df]), range(len(test_df)))

In [ ]:
emb_size = len(test_df)
D = torch.empty((emb_size, emb_size), device = 'cuda', dtype=torch.float16)

In [ ]:
generate_tfid_D(tfid_embs,D)
del tfid_embs

In [ ]:
D.mul_(-1)
D.add_(1)
gen_sim_and([img_embs, bert_embs],D)

In [ ]:
del img_embs, bert_embs

In [ ]:
rerank(D)

In [ ]:
edges = dist_to_edges(D)

In [ ]:
groups, groups_p = edges_to_groups(edges, len(D))
for pos, size_pct in get_targets_shape(train_df):
    chisel(groups, groups_p, pos, int(size_pct * len(groups)))

In [ ]:
matches = [' '.join(test_df.iloc[g].posting_id.to_list()) for g in groups]
test_df['matches'] = matches

test_df[['posting_id','matches']].to_csv('submission.csv',index=False)

In [ ]:
pd.read_csv('submission.csv').head()